In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_kddcup99
import torch
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
x = pd.read_csv("Reddit_Data.csv")
y = pd.read_csv('Twitter_Data.csv')
y = y.rename({'clean_text':'clean_comment'},axis=1)
x = x.append(y, ignore_index=True)

In [ ]:
x['category'] = x['category'].replace({1:'Positive',0:'Neutral',-1:'Negative'})

In [ ]:
sentences = np.array(x['clean_comment']).astype(str)
category = np.array(x['category']).astype(str)

In [ ]:
vect = CountVectorizer().fit(sentences)
X_train = vect.transform(sentences)

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, category)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(MultinomialNB(), X_train, category, cv=6)
print("Mean cross-validation accuracy: {:.2f}".format(np.mean(scores)))

In [ ]:
comment = np.array([input("Please enter a comment:\n")])
print('That comment is ',clf.predict(vect.transform(comment)[0]))

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_kddcup99
import torch
import numpy as np
import pandas as pd
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, X, y):
        'Initialization'
        self.labels = torch.tensor(y)
        self.features = torch.tensor(X)

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.labels)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample

        return self.features[index], self.labels[index]

In [3]:
x = pd.read_csv("Reddit_Data.csv")
y = pd.read_csv('Twitter_Data.csv')
y = y.rename({'clean_text':'clean_comment'},axis=1)
x = x.append(y, ignore_index=True).dropna()
sentences = np.array(x['clean_comment']).astype(str)
category = np.array(x['category'].astype('category').cat.codes).astype('int8')
vect = CountVectorizer().fit(sentences)
X_train_vect = vect.transform(sentences)
tf = TfidfTransformer().fit(X_train_vect)
X_train = tf.transform(X_train_vect).astype(np.float32)

In [27]:
import pickle
with open('textprocessing_models.pkl', 'wb') as fout:
    pickle.dump((vect, tf), fout)

In [4]:
X_train = X_train.toarray()

In [6]:
X_train.shape

(200118, 131695)

In [8]:

# Your answer goes here
X_train, X_test, y_train, y_test  = train_test_split(X_train, category, test_size=0.2)
training_set = Dataset(X_train, y_train)
training_generator = torch.utils.data.DataLoader(training_set,batch_size=256)
testing_set = Dataset(X_test, y_test)
testing_generator = torch.utils.data.DataLoader(testing_set,batch_size=64)

In [3]:
import torch
import torch.nn as nn


# Any Pytorch's network class is an extension of the torch.nn.Module parent class.
# To define a network class, you need to define at least 2 methods: an __init__() method (constructor) and a forward() method
class SimpleNetwork(nn.Module):
    # Create the network class by filling in this block of code

    # Create the constructor. Add any additional arguments as you wish
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNetwork,self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.reLu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size , num_classes)
    # Define the feed forward function.
    # x is the input example/examples.
    # Add any additional arguments as you wish.
    def forward(self, x):
        out = self.l1(x)
        out = self.reLu(out)
        out = self.l2(out)
        return out

In [17]:
import time
start = time.time()
#model = SimpleNetwork(131695, 100, 3)
end = time.time()
LEARNING_RATE = 0.0001
EPOCHS = 100
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = LEARNING_RATE)
n_total_steps = len(training_generator)
for epoch in range(EPOCHS):
    # Train the network by filling in this block of code
    model.train()
    train_loss = 0
    train_correct = 0
    start = time.time()
    for i,(xinput,youtput) in enumerate(training_generator):
        #xinput = xinput.to(device)
        #youtput = youtput.to(device)
        optimizer.zero_grad()
        outputs = model(xinput)
        loss = criterion(outputs,youtput.long())
        loss.backward()
        optimizer.step()
    #Record Train Accuracy
        train_loss += loss.item()
        train_pred = outputs.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += train_pred.eq(youtput.view_as(train_pred)).sum().item()
    train_loss /= len(training_generator.dataset)
    #Report Validation Accuracy
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testing_generator:
            output = model(data)
            test_loss += criterion(output, target.long()).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(testing_generator.dataset)
        end = time.time()
        print(f'epoch {epoch+1}/ {EPOCHS}')
        print('Time Taken: ',end - start)
        print('Training set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            train_loss, train_correct, len(training_generator.dataset),
            100. * train_correct / len(training_generator.dataset)))
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testing_generator.dataset),
            100. * correct / len(testing_generator.dataset)))

epoch 1/ 100
Time Taken:  112.17453050613403
Training set: Average loss: 0.0004, Accuracy: 157042/160094 (98%)

Test set: Average loss: 0.0036, Accuracy: 37074/40024 (93%)

epoch 2/ 100
Time Taken:  110.23135256767273
Training set: Average loss: 0.0004, Accuracy: 157047/160094 (98%)

Test set: Average loss: 0.0036, Accuracy: 37073/40024 (93%)



KeyboardInterrupt: 

In [16]:
torch.save(model.state_dict(), 'sentimentModel.pth')

In [4]:
sentimentModel = SimpleNetwork(131695, 100, 3)
sentimentModel.load_state_dict(torch.load('sentimentModel.pth'))
sentimentModel.eval()

SimpleNetwork(
  (l1): Linear(in_features=131695, out_features=100, bias=True)
  (reLu): ReLU()
  (l2): Linear(in_features=100, out_features=3, bias=True)
)

In [6]:
import pickle
with open('textprocessing_models.pkl', 'rb') as f:
    vect, tf = pickle.load(f)

In [7]:
test_loss = 0
correct = 0
with torch.no_grad():
        for data, target in testing_generator:
            output = sentimentModel(data.float())
            test_loss += criterion(output, target.long()).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(testing_generator.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testing_generator.dataset),
            100. * correct / len(testing_generator.dataset)))

NameError: name 'testing_generator' is not defined

In [12]:
import emoji
sentiment = {0:emoji.emojize(":pensive_face:"),
             1:emoji.emojize(":neutral_face:"),
             2:emoji.emojize(":smiling_face_with_smiling_eyes:")}
comment = np.array([input("Please enter a comment:\n")])
#print('That comment is ',clf.predict(vect.transform(comment)[0]))
prediction = sentimentModel(torch.tensor((tf.transform(vect.transform(comment)).toarray())).float())
# Predicted class value using argmax
print(sentiment[prediction.detach().numpy().argmax()])

Please enter a comment:
i hate this thing
😔
